<a href="https://colab.research.google.com/github/ravi72munde/scala-spark-cab-rides-predictions/blob/Ravi/Cab_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from google.colab import files

# uploaded = files.upload()

In [0]:
# !unzip cab_rides.zip

In [0]:
import pandas as pd

In [0]:
cab_df = pd.read_csv("./cab_rides.txt",delimiter='\t',encoding = "utf-8")
weather_df = pd.read_csv("./weather.csv")

In [3]:
cab_df['date_time'] = pd.to_datetime(cab_df['time_stamp']/1000, unit='s')
weather_df['date_time'] = pd.to_datetime(weather_df['time_stamp'], unit='s')
cab_df.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,date_time
0,1.08,Lyft,1.543280e+12,Northeastern University,Back Bay,10.5,1.0,462816a3-820d-408b-8549-0b39e82f65ac,lyft_plus,Lyft XL,2018-11-27 00:53:20
1,1.08,Lyft,1.543280e+12,Northeastern University,Back Bay,16.5,1.0,474d6376-bc59-4ec9-bf57-4e6d6faeb165,lyft_lux,Lux Black,2018-11-27 00:53:20
2,1.08,Lyft,1.543280e+12,Northeastern University,Back Bay,3.0,1.0,4f9fee41-fde3-4767-bbf1-a00e108701fb,lyft_line,Shared,2018-11-27 00:53:20
3,1.08,Lyft,1.543280e+12,Northeastern University,Back Bay,27.5,1.0,8612d909-98b8-4454-a093-30bd48de0cb3,lyft_luxsuv,Lux Black XL,2018-11-27 00:53:20
4,1.08,Lyft,1.543280e+12,Northeastern University,Back Bay,13.5,1.0,9043bf77-1d45-4a93-9520-a083e0277f16,lyft_premier,Lux,2018-11-27 00:53:20


In [0]:
cab_df['merge_date'] = cab_df.source.astype(str) +" - "+ cab_df.date_time.dt.date.astype("str") +" - "+ cab_df.date_time.dt.hour.astype("str")
weather_df['merge_date'] = weather_df.location.astype(str) +" - "+ weather_df.date_time.dt.date.astype("str") +" - "+ weather_df.date_time.dt.hour.astype("str")

In [0]:
weather_df.index = weather_df['merge_date']

In [0]:
# lyft_share_df = cab_df[cab_df.product_id=='lyft_line']
# cab_df = cab_df[cab_df.surge_multiplier	>1]

In [7]:
cab_df.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,date_time,merge_date
0,1.08,Lyft,1.543280e+12,Northeastern University,Back Bay,10.5,1.0,462816a3-820d-408b-8549-0b39e82f65ac,lyft_plus,Lyft XL,2018-11-27 00:53:20,Back Bay - 2018-11-27 - 0
1,1.08,Lyft,1.543280e+12,Northeastern University,Back Bay,16.5,1.0,474d6376-bc59-4ec9-bf57-4e6d6faeb165,lyft_lux,Lux Black,2018-11-27 00:53:20,Back Bay - 2018-11-27 - 0
2,1.08,Lyft,1.543280e+12,Northeastern University,Back Bay,3.0,1.0,4f9fee41-fde3-4767-bbf1-a00e108701fb,lyft_line,Shared,2018-11-27 00:53:20,Back Bay - 2018-11-27 - 0
3,1.08,Lyft,1.543280e+12,Northeastern University,Back Bay,27.5,1.0,8612d909-98b8-4454-a093-30bd48de0cb3,lyft_luxsuv,Lux Black XL,2018-11-27 00:53:20,Back Bay - 2018-11-27 - 0
4,1.08,Lyft,1.543280e+12,Northeastern University,Back Bay,13.5,1.0,9043bf77-1d45-4a93-9520-a083e0277f16,lyft_premier,Lux,2018-11-27 00:53:20,Back Bay - 2018-11-27 - 0


In [0]:
merged_df = cab_df.join(weather_df,on=['merge_date'],rsuffix ='_w')

In [0]:
merged_df['rain'].fillna(0,inplace=True)

In [0]:
merged_df = merged_df[pd.notnull(merged_df['date_time_w'])]

In [0]:
merged_df = merged_df[pd.notnull(merged_df['price'])]

In [0]:
merged_df['day'] = merged_df.date_time.dt.dayofweek

In [0]:
merged_df['hour'] = merged_df.date_time.dt.hour

In [14]:
merged_df['day'].describe()

count    920481.000000
mean          2.184254
std           1.568638
min           0.000000
25%           1.000000
50%           2.000000
75%           3.000000
max           6.000000
Name: day, dtype: float64

In [15]:
merged_df.columns

Index(['distance', 'cab_type', 'time_stamp', 'destination', 'source', 'price',
       'surge_multiplier', 'id', 'product_id', 'name', 'date_time',
       'merge_date', 'temp', 'location', 'clouds', 'pressure', 'time_stamp_w',
       'humidity', 'wind', 'rain', 'date_time_w', 'merge_date_w', 'day',
       'hour'],
      dtype='object')

In [0]:
# merged_df = merged_df.sample(frac =0.1)

In [17]:
merged_df.count()

distance            920481
cab_type            920481
time_stamp          920481
destination         920481
source              920481
price               920481
surge_multiplier    920481
id                  920481
product_id          920481
name                920481
date_time           920481
merge_date          920481
temp                920481
location            920481
clouds              920481
pressure            920481
time_stamp_w        920481
humidity            920481
wind                920481
rain                920481
date_time_w         920481
merge_date_w        920481
day                 920481
hour                920481
dtype: int64

In [0]:
X = merged_df[merged_df.product_id=='lyft_line'][['day','distance','hour','temp','clouds', 'pressure','humidity', 'wind', 'rain']]

In [559]:
X.count()

day         70966
distance    70966
hour        70966
temp        70966
clouds      70966
pressure    70966
humidity    70966
wind        70966
rain        70966
dtype: int64

In [0]:
y = merged_df['price'] 

In [527]:
y.count()

70966

In [0]:
X.reset_index(inplace=True)
X = X.drop(columns=['index'])

In [529]:
X.head()

,day,distance,hour,temp,clouds,pressure,humidity,wind,rain
0,1,1.08,0,43.96,1.00,1006.26,0.90,9.86,0.0497
1,1,1.08,0,43.83,0.97,1005.90,0.91,10.93,0.2173
2,1,1.08,0,43.82,0.97,1005.87,0.91,11.02,0.2039
3,1,1.08,0,43.82,0.97,1005.89,0.91,10.94,0.2154
4,1,1.08,0,43.98,1.00,1006.36,0.90,9.53,0.0358


In [0]:
features = pd.get_dummies(X)

In [531]:
features.columns

Index(['day', 'distance', 'hour', 'temp', 'clouds', 'pressure', 'humidity',
       'wind', 'rain'],
      dtype='object')

In [0]:
# Use numpy to convert to arrays
import numpy as np
# Labels are the values we want to predict
labels = np.array(y)

# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [0]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [534]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (53224, 9)
Training Labels Shape: (53224,)
Testing Features Shape: (17742, 9)
Testing Labels Shape: (17742,)


In [0]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
# Instantiate model with 100 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [536]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.38 degrees.


In [537]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 92.79 %.


In [538]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: distance             Importance: 0.71
Variable: temp                 Importance: 0.08
Variable: pressure             Importance: 0.07
Variable: wind                 Importance: 0.04
Variable: hour                 Importance: 0.03
Variable: clouds               Importance: 0.02
Variable: humidity             Importance: 0.02
Variable: rain                 Importance: 0.02
Variable: day                  Importance: 0.01


**Lets see the prediction for surge_multiplier**

In [0]:
merged_df_surge = merged_df[merged_df.surge_multiplier < 3]
X = merged_df_surge[['day','hour','temp','clouds', 'pressure','humidity', 'wind', 'rain']]

In [19]:
X.count()

day         920479
hour        920479
temp        920479
clouds      920479
pressure    920479
humidity    920479
wind        920479
rain        920479
dtype: int64

In [0]:
features = pd.get_dummies(X)

In [0]:
# # Use numpy to convert to arrays
# import numpy as np
# # Labels are the values we want to predict
# labels = np.array(y)

# # Saving feature names for later use
# feature_list = list(X.columns)
# # Convert to numpy array
# features = np.array(features)

In [0]:
y = merged_df_surge['surge_multiplier']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit([1,1.25,1.5,1.75,2.,2.25,2.5])
y = le.transform(y) 

In [0]:
# Use numpy to convert to arrays
import numpy as np
# Labels are the values we want to predict
labels = np.array(y)

# Saving feature names for later use
feature_list = list(X.columns)
# Convert to numpy array
features = np.array(features)

In [0]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [0]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
train_features, train_labels = sm.fit_resample(train_features, train_labels)

In [30]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state = 42,class_weight="balanced")
# Train the model on training data
rf.fit(train_features, train_labels);

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [31]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.41 degrees.


In [64]:
from sklearn.metrics import precision_score, recall_score
precision_score(test_labels, predictions, average="weighted")


0.9824943160843629

In [65]:
recall_score(test_labels, predictions, average="micro")

0.7769468103598123

In [40]:
# Create confusion matrix
pd.crosstab(le.inverse_transform(test_labels), le.inverse_transform(predictions),rownames=['Actual'],colnames=['Predicted'])

Predicted,1.0,1.25,1.5,1.75,2.0,2.5
Actual,,,,,,
1.00,176822,25868,10898,6693,5436,407
1.25,141,1354,439,403,222,28
1.50,5,188,333,200,110,11
1.75,0,37,44,143,40,8
2.00,6,11,35,43,133,46
2.50,1,1,3,2,3,6


In [0]:
import matplotlib.pyplot as plt


In [45]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: temp                 Importance: 0.19
Variable: pressure             Importance: 0.18
Variable: wind                 Importance: 0.18
Variable: humidity             Importance: 0.14
Variable: clouds               Importance: 0.09
Variable: hour                 Importance: 0.08
Variable: rain                 Importance: 0.07
Variable: day                  Importance: 0.06


In [0]:
from sklearn.metrics import accuracy_score

In [35]:
accuracy_score(test_labels, predictions)

0.7769468103598123